<a href="https://colab.research.google.com/github/nduc4nh/CNNResnet_Music_Classification/blob/main/Workspace_Conv_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install unrar

In [ ]:
!unrar x '/content/drive/MyDrive/Genre Data/sample_data.rar'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/Genre Data/sample_data.rar

Extracting  cheo                                                           0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24%  OK 
Extracting  quanho                                                        24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50%  OK 
Extracting  xam                                                           50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75

In [ ]:
!pip install pickle5
import pickle5

In [ ]:
with open('/content/cheo','rb') as f:
  cheo = pickle5.load(f)
cheo = cheo.reshape([cheo.shape[0],cheo.shape[2],cheo.shape[1]])

In [ ]:
with open('/content/quanho','rb') as f:
  quanho = pickle5.load(f)
quanho = quanho.reshape([quanho.shape[0],quanho.shape[2],quanho.shape[1]])

In [ ]:
with open('/content/xam','rb') as f:
  xam = pickle5.load(f)
xam = xam.reshape([xam.shape[0],xam.shape[2],xam.shape[1]])

In [ ]:
with open('/content/Catru','rb') as f:
  catru = pickle5.load(f)
catru = catru.reshape([catru.shape[0],catru.shape[2],catru.shape[1]])

In [ ]:
import numpy as np

In [ ]:
y_cheo = np.ones((cheo.shape[0],))
y_quanho = np.ones((quanho.shape[0],))*3/4
y_xam = np.ones((xam.shape[0],))*1/2
y_catru = np.ones((catru.shape[0],))*1/4


In [ ]:
y_cheo1 = np.ones((cheo.shape[0],))
y_quanho1 = np.ones((quanho.shape[0],))*2
y_xam1 = np.ones((xam.shape[0],))*3
y_catru1 = np.ones((catru.shape[0],))*4


In [ ]:
X = np.concatenate([cheo,quanho,xam,catru])
y = np.concatenate([y_cheo,y_quanho,y_xam,y_catru])
y1 = np.concatenate([y_cheo1,y_quanho1,y_xam1,y_catru1])

In [ ]:
del cheo, quanho, xam, catru

In [ ]:
l = X.shape[0]
train_l = l*1//4
train_idx = [i for i in range(train_l)]
valid_idx = [i for i in range(train_l,2*train_l)]
test_idx = [i for i in range(2*train_l,l)]

In [ ]:
tmp = [i for i in range(l)]
np.random.shuffle(tmp)

In [ ]:
X = X[tmp,:]
y = y[tmp]
y1 = y1[tmp]

In [ ]:
X_train = X[train_idx,:]
X_valid = X[valid_idx,:]
X_test = X[test_idx,:]

In [ ]:
y_train = y[train_idx]
y_valid = y[valid_idx]
y_test = y[test_idx]
y_train1 = y1[train_idx]
y_valid1 = y1[valid_idx]
y_test1 = y1[test_idx]

In [ ]:
def make_one_hot(x,num = 4):
  return [0 if i != int(x) - 1 else 1 for i in range(num)]

In [ ]:
y_train_1hot = np.array([make_one_hot(ele) for ele in y_train1])
y_test_1hot = np.array([make_one_hot(ele) for ele in y_test1])
y_valid_1hot = np.array([make_one_hot(ele) for ele in y_valid1])


In [ ]:
y_train

array([0.5 , 1.  , 0.5 , ..., 1.  , 0.5 , 0.25])

In [ ]:
del X,y

In [ ]:
del X_test

##One-hot Model 11


In [ ]:
class Dropout(layers.Dropout):
  def get_config(self):
        config = super().get_config().copy()
        config.update({
            'rate': 0.5,
        })
        return config

In [ ]:
model_1hot = tf.keras.Sequential(
      [
        layers.Conv1D(64,3,3),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.Conv1D(64,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(64,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(128,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(128,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(128,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(256,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Flatten(),
        Dropout(0.5),
        layers.Dense(4, activation='softmax'),
      ]
)



In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model_1hot.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:

model_1hot.build(input_shape=(None,110250,1))


In [ ]:
model_1hot.save('model_h5_conv1d_11.h5')

In [ ]:
model_1hot.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 36750, 64)         256       
_________________________________________________________________
batch_normalization (BatchNo (None, 36750, 64)         256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 36750, 64)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 36748, 64)         12352     
_________________________________________________________________
batch_normalization_1 (Batch (None, 36748, 64)         256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 36748, 64)         0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 12249, 64)         0

In [ ]:
model_1hot.fit(X_train,y_train_1hot,validation_data=(X_valid, y_valid_1hot),batch_size = 64,epochs = 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/50
42/42 [==============================] - 69s 882ms/step - loss: 7.9445 - accuracy: 0.2966 - val_loss: 68.4735 - val_accuracy: 0.2750
Epoch 2/50
42/42 [==============================] - 36s 866ms/step - loss: 3.7437 - accuracy: 0.3841 - val_loss: 22.8570 - val_accuracy: 0.2341
Epoch 3/50
42/42 [==============================] - 37s 889ms/step - loss: 3.0295 - accuracy: 0.4298 - val_loss: 4.9090 - val_accuracy: 0.2698
Epoch 4/50
42/42 [==============================] - 37s 896ms/step - loss: 2.0930 - accuracy: 0.5326 - val_loss: 2.9247 - val_accuracy: 0.2635
Epoch 5/50
42/42 [==============================] - 37s 887ms/step - loss: 1.5306 - accuracy: 0.5932 - val_loss: 6.5185 - val_accuracy: 0.2560
Epoch 6/50
42/42 [==============================] - 40s 967ms/step - loss: 1.1261 - accuracy: 0.6729 - val_loss: 13.4346 - val_accuracy: 0.2557
Epoch 7/50
42/42 [==============================] - 37s 891ms/step - loss: 0.8724 - accuracy: 0.7097 - val_loss: 6.9611 - val_accuracy: 0.2

In [ ]:
mod

In [ ]:
model = tf.keras.Sequential(
      [
        layers.Conv1D(64,3,3),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.Conv1D(64,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(64,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(128,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(128,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(128,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Conv1D(256,3,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(3,3),
        layers.Flatten(),
        keras.layers.Dropout(0.5),
        layers.Dense(4, activation='softmax')
      ]
)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0000001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
model.build(input_shape=(None,110250,1))

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train1,validation_data=(X_valid, y_valid),batch_size = 64,epochs = 100)

Epoch 1/100
20/42 [=============>................] - ETA: 21s - loss: nan - accuracy: 0.0086

KeyboardInterrupt: ignored

In [ ]:
X_train.mean()

-3.5162725e-06

In [ ]:
model.fit()

In [ ]:
model.evaluate(X_test,y_test,verbose=2)

In [ ]:
#model.save("/content/drive/MyDrive/Genre Data/model_h5_conv1d.h5")

In [ ]:
#model.predict(X_train)[0]

array([2.0142358e-11, 3.8968664e-02, 2.4016083e-05, 9.5911556e-01,
       1.8917950e-03, 1.7479967e-11, 1.9667205e-11, 2.3937716e-11,
       2.3298967e-11, 2.5884420e-11], dtype=float32)

## MODEL 8

In [ ]:
model8 = tf.keras.Sequential(
      [
        layers.Conv1D(1,256,256),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.Conv1D(32,8,2),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.Conv1D(32,8,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(4,4),
        layers.Conv1D(32,8,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(4,4),
        layers.Conv1D(32,8,1),layers.BatchNormalization(),layers.LeakyReLU(),
        layers.MaxPool1D(4,4),
        layers.LSTM(units=64),
        layers.Flatten(),
        Dropout(0.5),
        layers.Dense(4, activation='softmax')
      ]
)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model8.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
model8.build(input_shape=(None,110250,1))

In [ ]:
model8.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_38 (Conv1D)           (None, 430, 1)            257       
_________________________________________________________________
batch_normalization_38 (Batc (None, 430, 1)            4         
_________________________________________________________________
leaky_re_lu_38 (LeakyReLU)   (None, 430, 1)            0         
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 212, 32)           288       
_________________________________________________________________
batch_normalization_39 (Batc (None, 212, 32)           128       
_________________________________________________________________
leaky_re_lu_39 (LeakyReLU)   (None, 212, 32)           0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 205, 32)          

In [ ]:
model8.fit(X_train,y_train_1hot,validation_data=(X_valid, y_valid_1hot),batch_size = 64,epochs = 100)

Epoch 1/100
42/42 [==============================] - 5s 115ms/step - loss: 0.6194 - accuracy: 0.7693 - val_loss: 1.2495 - val_accuracy: 0.4771
Epoch 2/100
42/42 [==============================] - 5s 114ms/step - loss: 0.6200 - accuracy: 0.7670 - val_loss: 1.2508 - val_accuracy: 0.4749
Epoch 3/100
42/42 [==============================] - 5s 113ms/step - loss: 0.6079 - accuracy: 0.7782 - val_loss: 1.2541 - val_accuracy: 0.4745
Epoch 4/100
42/42 [==============================] - 5s 115ms/step - loss: 0.6059 - accuracy: 0.7830 - val_loss: 1.2533 - val_accuracy: 0.4760
Epoch 5/100
42/42 [==============================] - 5s 115ms/step - loss: 0.6117 - accuracy: 0.7644 - val_loss: 1.2626 - val_accuracy: 0.4723
Epoch 6/100
42/42 [==============================] - 5s 116ms/step - loss: 0.6050 - accuracy: 0.7789 - val_loss: 1.2674 - val_accuracy: 0.4715
Epoch 7/100
42/42 [==============================] - 5s 114ms/step - loss: 0.6082 - accuracy: 0.7793 - val_loss: 1.2567 - val_accuracy: 0.4756

In [ ]:
model8.save('model_h5_conv1d_8.h5')

## RESNET


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import keras
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [ ]:
def residual_block_(filters, size, stride):
  def f(x):
    y = keras.layers.ZeroPadding1D(2)(x)
    y = keras.layers.Conv1D(filters, size, stride)(y)
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.LeakyReLU()(y)
    y = keras.layers.Conv1D(filters, size, stride)(y)
    y = keras.layers.BatchNormalization()(y)

    x = keras.layers.Conv1D(filters, 1, stride)(x)
    print(x.shape)
    print(y.shape)
    y = keras.layers.Add()([x,y])

  return f

In [ ]:
x = np.random.rand(2,110250,1)
y = np.random.rand(20,100,128)

In [ ]:
residual_block_(128,3,1)(x)

(2, 110250, 128)
(2, 110250, 128)


In [ ]:
class ResidualBlock(keras.layers.Layer):
  def __init__(self, filters, kernel, stride):
    super(ResidualBlock, self).__init__()
    
    """self.body = keras.models.Sequential(
        [
          keras.layers.Conv1D(filters, kernel, stride),
          keras.layers.BatchNormalization(),
          keras.layers.LeakyReLU(),
          keras.layers.Conv1D(filters, kernel, stride),
          keras.layers.BatchNormalization()
        ]
    )"""

    self.filters = filters
    self.kernel = kernel
    self.stride = stride
  
  def build(self, input_shape):
    self.body = keras.models.Sequential(
        [
          keras.layers.Conv1D(self.filters, self.kernel, self.stride),
          keras.layers.BatchNormalization(),
          keras.layers.LeakyReLU(),
          keras.layers.Conv1D(self.filters, self.kernel, self.stride),
          keras.layers.BatchNormalization()
        ]
      )   
    super(ResidualBlock, self).build(input_shape) 
    
  def call(self, inputs):
    x = keras.layers.Conv1D(self.filters, 1, self.stride)(inputs)
    y = keras.layers.ZeroPadding1D(2)(inputs)
    y = self.body(y)
    y = keras.layers.Add()([x,y])
    y = keras.layers.LeakyReLU()(y)
    return y

  def get_config(self):

        config = super().get_config().copy()
        config.update({
            'filters': self.filters,
            'kernel': self.kernel,
            'stride': self.stride
        })
        return config

In [ ]:
model_res_conv1d = keras.models.Sequential(
    [
     keras.layers.Conv1D(128,3,3),
     ResidualBlock(128,3,1),
     keras.layers.MaxPooling1D(3,3),
     ResidualBlock(128,3,1),
     keras.layers.MaxPooling1D(3,3),
     ResidualBlock(256,3,1),
     keras.layers.MaxPooling1D(3,3),
     ResidualBlock(256,3,1),
     keras.layers.MaxPooling1D(3,3),
     ResidualBlock(256,3,1),
     keras.layers.MaxPooling1D(3,3),
     ResidualBlock(256,3,1),
     keras.layers.MaxPooling1D(3,3),
     ResidualBlock(256,3,1),
     keras.layers.MaxPooling1D(3,3),
     ResidualBlock(256,3,1),
     keras.layers.MaxPooling1D(3,3),
     ResidualBlock(512,3,1),
     keras.layers.MaxPooling1D(3,3),
     keras.layers.Conv1D(512,1,1),
     keras.layers.Flatten(),
     keras.layers.Dense(4, activation="softmax")
    ]
)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model_res_conv1d.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
model_res_conv1d.build(input_shape=(None, 110250,1))

In [ ]:
model_res_conv1d.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 36750, 128)        512       
_________________________________________________________________
residual_block (ResidualBloc (None, 36750, 128)        99584     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 12250, 128)        0         
_________________________________________________________________
residual_block_1 (ResidualBl (None, 12250, 128)        99584     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4083, 128)         0         
_________________________________________________________________
residual_block_2 (ResidualBl (None, 4083, 256)         297472    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1361, 256)         0

In [ ]:
model_res_conv1d.fit(X_train,y_train_1hot,validation_data=(X_valid, y_valid_1hot),batch_size = 10,epochs = 100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/100
269/269 [==============================] - 331s 1s/step - loss: 2.5291 - accuracy: 0.2806 - val_loss: 1.7234 - val_accuracy: 0.2639
Epoch 2/100
269/269 [==============================] - 295s 1s/step - loss: 1.8648 - accuracy: 0.3658 - val_loss: 1.8420 - val_accuracy: 0.2933
Epoch 3/100
269/269 [==============================] - 295s 1s/step - loss: 1.6471 - accuracy: 0.3905 - val_loss: 1.8550 - val_accuracy: 0.3405
Epoch 4/100
269/269 [==============================] - 295s 1s/step - loss: 1.4814 - accuracy: 0.4163 - val_loss: 1.7172 - val_accuracy: 0.3662
Epoch 5/100
269/269 [==============================] - 295s 1s/step - loss: 1.3082 - accuracy: 0.4336 - val_loss: 1.7204 - val_accuracy: 0.3710
Epoch 6/100
269/269 [==============================] - 295s 1s/step - loss: 1.2657 - accuracy: 0.4524 - val_loss: 1.4273 - val_accuracy: 0.3982
Epoch 7/100
269/269 [==============================] - 308s 1s/step - loss: 1.1955 - accuracy: 0.4828 - val_loss: 1.6304 - val_accuracy:

In [ ]:
#for heavy model architectures, .h5 file is unsupported.
#Save model
weigh= model_res_conv1d.get_weights();    pklfile= "/content/drive/MyDrive/model_resnet_pickle"
try:
    fpkl= open(pklfile, 'wb')    #Python 3     
    pickle5.dump(weigh, fpkl, protocol= pickle5.HIGHEST_PROTOCOL)
    fpkl.close()
except:
    fpkl= open(pklfile, 'w')    #Python 2      
    pickle5.dump(weigh, fpkl, protocol= pickle5.HIGHEST_PROTOCOL)
    fpkl.close()

In [ ]:
#load model
with open(r'/content/drive/MyDrive/model_resnet_pickle','rb') as f:
  df = pickle5.load(f)
model_res_conv1d.set_weights(df)

##TEST


In [ ]:
def sliding_window(y,sr,size = 5,stride = 3):
    sr_size = sr*size
    step = sr*stride
    
    start = 0
    re = []
    n = len(y)
    tmp = np.zeros(((n - (size - stride))//(sr*stride),1,22050*5),dtype = 'float32')
    i = 0
    while start + sr_size < n:
        tmp[i,:,:] = y[start:start + sr_size]
        start += step
        i += 1
    return tmp

In [ ]:
import librosa

In [ ]:
path = '/content/HatXam-XuanHinh_arka.mp3'
y ,sr = librosa.load(path)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
t = sliding_window(y ,sr)
t = t.reshape([t.shape[0],t.shape[2],t.shape[1]])

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [ ]:
np.sum(model_res_conv1d.predict(t),axis = 0)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


array([ 3.3643072, 37.249153 , 36.404358 , 10.982186 ], dtype=float32)

In [ ]:
t = t.reshape([t.shape[0],t.shape[2],t.shape[1]])

In [ ]:
t.shape

In [ ]:
import torch


In [ ]:
s = torch.nn.Conv1d(1,64,3,3)(torch.Tensor(t))

In [ ]:
s.shape

In [ ]:
X_valid.std()

In [ ]:
X_valid.mean()

In [ ]:
X_valid.mean()

In [ ]:
y.mean()

In [ ]:
y.shape

In [ ]:
from sklearn.preprocessing import RobustScaler,Normalizer

In [ ]:
l = Normalizer().fit_transform(y.reshape(y.shape[0],1))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(l, bins = 100)
plt.show()

In [ ]:
l.max()

In [ ]:
l.std()/l.mean()

In [ ]:
y.std()

In [ ]:
y.std()/y.mean()

In [ ]:
plt.hist(y, bins = 100)
plt.show()

In [ ]:
plt.plot(y)

In [ ]:
tmp = torch.nn.Conv1d(1,1,256,256)(torch.Tensor(y.reshape(1,1,y.shape[0])))

In [ ]:
tmp.mean()

In [ ]:
tmp = keras.models.load_model(r'./model_1hot.h5')

ValueError: ignored

In [ ]:
model_res_conv1d